In [31]:
#Old API download

import requests
import json
import re
import os

#0. Variables needed (list of genes, tax id and the database file name)
gene_list_path = "gene_list/enamelome_JK.txt"
db_name = "proboscidea_db.fasta"
tax_id = 9779

#1. Create the gene list
gene_list = []

with open(gene_list_path, "rt") as gene_list_file:
    for gene in gene_list_file:
        gene_list.append(gene[0:-1])

#2. Create the taxID list of species contained in the taxID

import requests
import os

tax_id = 9779
url_tax_id_descendent = "https://rest.uniprot.org/taxonomy/stream?format=list&query=%28ancestor%3A" + str(tax_id) + "%29"

tax_id_list = []

with open("./taxID.txt", "wb") as tax_id_descendent_file:
            
    #Extract each protein record for the search
    with requests.get(url_tax_id_descendent, stream=True) as request_tax_id_descedent:
        request_tax_id_descedent.raise_for_status()

        #Append the protein records to the db fasta file
        for id in request_tax_id_descedent.iter_content():
            tax_id_descendent_file.write(id)


with open("./taxID.txt", "rt") as tax_id_file:
    for id in tax_id_file:
        tax_id_list.append(int(id[0:-1]))

os.remove("./taxID.txt") 

#taxID organism name

import requests
import os
import json

tax_id = 9779
url_org_name = "https://rest.uniprot.org/taxonomy/stream?format=json&query=%28" + str(tax_id) + "%29"

with open("./org_name.json", "wb") as org_name_json:
            
    #Extract each protein record for the search
    with requests.get(url_org_name, stream=True) as request_org_name:
        request_org_name.raise_for_status()

        #Append the protein records to the db fasta file
        for chunk in request_org_name.iter_content():
            org_name_json.write(chunk)

with open("./org_name.json", "rt") as org_name_json:
    org_name_dic = json.load(org_name_json)

    org_name = org_name_dic["results"][0]["scientificName"]

os.remove("./org_name.json")

#3. Convert JSON format to FASTA

def json_to_fasta(json_path, fasta_path):
    with open(json_path, "rt") as prot_json:
        prot_dic = json.load(prot_json) #Convert json format into a dictionary

    with open(fasta_path, "at") as prot_fasta: #Write in a text file the fasta records
        for record in prot_dic["results"]:
            
            #Construct the header
            header = ">" + record["uniParcCrossReferences"][0]["database"] + "|" + record["uniParcId"] + " " + \
            record["uniParcCrossReferences"][0]["proteinName"] + " "
            
            if record["uniParcCrossReferences"][0]["organism"]["taxonId"] not in tax_id_list:
                header += "OS=" + org_name + " " + \
                "OX=" + str(tax_id) + " "
            else:
                header += "OS=" + record["uniParcCrossReferences"][0]["organism"]["scientificName"] + " " + \
                "OX=" + str(record["uniParcCrossReferences"][0]["organism"]["taxonId"]) + " "
            
            header += "GN=" + gene

            prot_fasta.write(header + "\n")

            #Write the protein sequence in rows of 60 aminoacids
            sequence_split = re.findall(".{1,60}", record["sequence"]["value"])
            for row in sequence_split:
                prot_fasta.write(row + "\n")

    prot_json.close()

#4. Programmatic acces to UniProtKb through the UniProt API  

#remove the previous database with the same name

if os.path.exists("proboscidea_db.fasta"):
    os.remove("proboscidea_db.fasta")

for gene in gene_list: 
    #API URL needed to download the proper sequences
    url = "https://rest.uniprot.org/uniparc/stream?compressed=false&format=json&query=%28%28gene%3A" + gene + "%29%20AND%20%28taxonomy_id%3A" + str(tax_id) + "%29%29"

    #Create the db fasta file to append the protein records
    with open("./db.json", "wb") as db_json:
            
            #Extract each protein record for the search
            with requests.get(url, stream=True) as request:
                request.raise_for_status()

                #Append the protein records to the db fasta file
                for chunk in request.iter_content():
                    db_json.write(chunk)
            
    db_json.close()

    json_to_fasta("./db.json", db_name)
    os.remove("./db.json")

In [14]:
# Internet connection

import requests

def internet_on():
    try:
        requests.get('http://www.google.com', timeout=5)
        return True
    except requests.ConnectionError as err: 
        return False

if not internet_on():
    print("ERROR: No internet connection")
    exit(0)

ERROR: No internet connection


: 

In [26]:
#New API downolad full
import requests
import json
import re
import os
from requests.adapters import HTTPAdapter, Retry

# Script information - Written in Python 3.9.12 - April 2023
__author__ = "Guillermo Carrillo Martín"
__maintainer__ = "Guillermo Carrillo Martín"
__email__ = "guillermo.carrillo@upf.edu"

"""
This script creates a protein database in fasta format using the UniParc 
archive, a non redundand database that contains all the proteins annotated 
using ENSEMBL, NCBI and other resouces. The search is focus for a specific 
taxonomic group (indicated using the NCBI tax ID) and can be reduced to 
a certain group of genes, indicated in a list. Other spcecificities, as 
the descripion of the protein header, can be seen in the README.md file.
"""

#0. Variables needed
gene_list_path = "gene_list/enamelome_JK.txt" # The path to the list of genes
db_name = "proboscidea_db.fasta" # The name of the database file
tax_id = 9779 # The tax ID of the organism used to search in UniParc

#1. Create the gene list
gene_list = []

with open(gene_list_path, "rt") as gene_list_file: #Open gene list file
    for gene in gene_list_file:
        gene_list.append(gene[0:-1]) #Append the genes in the file to a list

#2. Create a taxID list of taxonomic descendent species contained in the taxID

"""
In Uniprot, all the proteins with the same sequence have the same unic
identifier even they belong to different species. So, at the end, if a 
protein is conserved throught many species it appears in the database as
one record (and the proper unic idenifier) with many species associated.
Because of this, sometimes the program retrieves in a recorda random specie 
that is not from our linage. To fix this error, we create a list with all the 
downstream taxID to evaluate if the association belongs or not to the correct
linage. If it founds a missmatch, in the header there will be the following
modifications:

OS= taxID corresponging name (for example, Proboscidea)
OX= taxID number (for example, 9779)
"""

# 2.1 Build API link for the taxonomy download in uniprot
url_tax_id_descendent = "https://rest.uniprot.org/taxonomy/stream?format=list&query=%28ancestor%3A" + str(tax_id) + "%29"
tax_id_list = []

# Download the taxID's in a temporal text file
with open("./taxID.txt", "wb") as tax_id_descendent_file:
    
    # Extract each taxID from the search        
    with requests.get(url_tax_id_descendent, stream=True) as request_tax_id_descedent:
        request_tax_id_descedent.raise_for_status()

        # Append the taxID's in the text file
        for id in request_tax_id_descedent.iter_content():
            tax_id_descendent_file.write(id)

# Open the taxID's file and append them into a list 
with open("./taxID.txt", "rt") as tax_id_file:
    for id in tax_id_file:
        tax_id_list.append(int(id[0:-1]))

# Remove the temporal text file with the taxID's
os.remove("./taxID.txt") 

# 2.2 taxID organism name

tax_id = 9779
url_org_name = "https://rest.uniprot.org/taxonomy/stream?format=json&query=%28" + str(tax_id) + "%29"

with open("./org_name.json", "wb") as org_name_json:
            
    #Extract each protein record for the search
    with requests.get(url_org_name, stream=True) as request_org_name:
        request_org_name.raise_for_status()

        #Append the protein records to the db fasta file
        for chunk in request_org_name.iter_content():
            org_name_json.write(chunk)

with open("./org_name.json", "rt") as org_name_json:
    org_name_dic = json.load(org_name_json)

    org_name = org_name_dic["results"][0]["scientificName"]

os.remove("./org_name.json")

#3. Convert JSON format to FASTA

def json_to_fasta(json_path, fasta_path, gene_name):
    with open(json_path, "rt") as prot_json:
        prot_dic = json.load(prot_json) #Convert json format into a dictionary

    with open(fasta_path, "at") as prot_fasta: #Write in a text file the fasta records
        for record in prot_dic["results"]:
            
            #Construct the header
            header = ">" + record["uniParcCrossReferences"][0]["database"] + "|" + record["uniParcId"] + "|" + \
            record["uniParcCrossReferences"][0]["lastUpdated"]+ " " + \
            record["uniParcCrossReferences"][0]["proteinName"] + " "
            
            if record["uniParcCrossReferences"][0]["organism"]["taxonId"] not in tax_id_list:
                header += "OS=" + org_name + " " + \
                "OX=" + str(tax_id) + " "
            else:
                header += "OS=" + record["uniParcCrossReferences"][0]["organism"]["scientificName"] + " " + \
                "OX=" + str(record["uniParcCrossReferences"][0]["organism"]["taxonId"]) + " "
            
            header += "GN=" + gene_name

            prot_fasta.write(header + "\n")

            #Write the protein sequence in rows of 60 aminoacids
            sequence_split = re.findall(".{1,60}", record["sequence"]["value"])
            for row in sequence_split:
                prot_fasta.write(row + "\n")

    prot_json.close()


#4. Programmatic acces to UniProtKb through the UniProt API  

#remove the previous database with the same name

if os.path.exists("proboscidea_db.fasta"):
    os.remove("proboscidea_db.fasta")

#Programatic acces setup

re_next_link = re.compile(r'<(.+)>; rel="next"')
retries = Retry(total=5, backoff_factor=0.25, status_forcelist=[500, 502, 503, 504])
session = requests.Session()
session.mount("https://", HTTPAdapter(max_retries=retries))

def get_next_link(headers):
    if "Link" in headers:
        match = re_next_link.match(headers["Link"])
        if match:
            return match.group(1)

def get_batch(batch_url):
    while batch_url:
        response = session.get(batch_url)
        response.raise_for_status()
        total = response.headers["x-total-results"]
        yield response, total
        batch_url = get_next_link(response.headers)



#Create the db fasta file to append the protein records
progress = 0

for gene in gene_list:
    #API URL needed to download the proper sequences
    url = "https://rest.uniprot.org/uniparc/search?compressed=false&format=json&query=%28%28gene%3A" + gene + "%29%20AND%20%28taxonomy_id%3A" + str(tax_id) + "%29%29&size=500"

    with open("./db.json", 'w') as f:
        for batch, total in get_batch(url):
            lines = batch.text.splitlines()
            if not progress:
                print(lines[0], file=f)
            for line in lines[1:]:
                print(line, file=f)
            progress += len(lines[1:])

    json_to_fasta("./db.json", db_name, gene)
    os.remove("./db.json")

In [ ]:
#Remove duplicates

#Global imports
import re
import os

#Variables
input_file_path = "proboscidea_db.fasta"
output_file_path = "proboscidea_db_removed.fasta"

#Remove breaks at the end of the sequence lines
def fasta_oneliner(input_path, output_path):
    fasta_file = open(input_path, "rt")
    output = open(output_path, "wt")

    lin = fasta_file.readline()
    output.write(lin)

    for line in fasta_file:
        match = re.search(r"^>",line)
        if match:
            output.write("\n" + line)
        else:
            output.write(line.replace("\n",""))

    fasta_file.close()
    output.close()

fasta_oneliner(input_file_path, "./oneliner.fasta")

#Append the records to a nested list [header, sequence]
record_list = []

with open("./oneliner.fasta", "rt") as multifasta_file:
    while True:
        tag = multifasta_file.readline().strip() #Tag
        
        if not tag: 
            break #End of file
        
        sequence = multifasta_file.readline().strip() #Sequence

        record = [tag, sequence]

        record_list.append(record)

#Remove the temporal sequence-oneliner file
os.remove("./oneliner.fasta")

#Create a new list to remove the sequences
record_list_removed = record_list.copy()

#Create variables to count the number of removed records
records_equal_count = 0 
records_contained_count = 0

#Remove the duplicates in the list
for record1 in record_list_removed:
    for record2 in record_list_removed[record_list_removed.index(record1) + 1:]:
        if record1[1] == record2[1]:
            record_list_removed.remove(record1)
            records_equal_count += 1
            break

        elif record1[1] in record2[1]:
            record_list_removed.remove(record1)
            records_contained_count += 1
            break

        elif record2[1] in record1[1]:
            record_list_removed.remove(record2)
            records_contained_count += 1

print(len(record_list))
print(len(record_list_removed))

#Writte the records in a new file
with open(output_file_path, "wt") as output:
    for record in record_list:
        if record in record_list_removed:
            output.write(record[0] + "\n")

            sequence_split = re.findall(".{1,60}", record[1])
            for row in sequence_split:
                output.write(row + "\n")

#Write the duplicate records in a new file
with open("./duplicate_records", "wt") as duplicate_records_file:
    for record in record_list:
        if record not in record_list_removed:
            duplicate_records_file.write(record[0] + "\n")

            sequence_split = re.findall(".{1,60}", record[1])
            for row in sequence_split:
                duplicate_records_file.write(row + "\n")

# Print the removed number of records
if records_equal_count == 1:
    print("%d record with the same sequence has been removed" % records_equal_count)
else:
    print("%d records with the same sequence have been removed" % records_equal_count)

if records_contained_count == 1:
    print("%d fragment records has been removed" % records_contained_count)
else:
    print("%d fragment records have been removed" % records_contained_count)

In [ ]:
#Alternative API download

# Global imports
import json
import re
import os
import requests
from requests.adapters import HTTPAdapter, Retry

# Script information - Written in Python 3.9.12 - April 2023
__author__ = "Guillermo Carrillo Martín"
__maintainer__ = "Guillermo Carrillo Martín"
__email__ = "guillermo.carrillo@upf.edu"

"""
This script creates a protein database in fasta format using the UniParc 
archive, a non redundand database that contains all the proteins annotated 
using ENSEMBL, NCBI and other resouces. The search is focus for a specific 
taxonomic group (indicated using the NCBI tax ID) and can be reduced to 
a certain group of genes, indicated in a list. Other spcecificities, as 
the descripion of the protein header, can be seen in the README.md file.
"""

#0. Variables needed
gene_list_path = "./gene_list/enamelome_JK.txt" # The path to the list of genes
db_name = "proboscidea_db.fasta" # The name of the database file
tax_id = 9779 # The tax ID of the organism used to search in UniParc

#1. Create the gene list
if "gene_list_path" in globals():
    gene_list = []

    with open(gene_list_path, "rt") as gene_list_file: #Open gene list file
        for gene in gene_list_file:
            gene_list.append(gene[0:-1]) #Append the genes in the file to a list

#2. Create a taxID list of taxonomic descendent species contained in the taxID

"""
In Uniprot, all the proteins with the same sequence have the same unic
identifier even they belong to different species. So, at the end, if a 
protein is conserved throught many species it appears in the database as
one record (and the proper unic idenifier) with many species associated.
Because of this, sometimes the program retrieves in a recorda random specie 
that is not from our linage. To fix this error, we create a list with all the 
downstream taxID to evaluate if the association belongs or not to the correct
linage. If it founds a missmatch, in the header there will be the following
modifications:

OS= taxID corresponging name (for example, Proboscidea)
OX= taxID number (for example, 9779)
"""

# 2.1 Build API link for the taxonomy download in uniprot
url_tax_id_descendent = "https://rest.uniprot.org/taxonomy/stream?format=list&query=%28%28ancestor%3A{}%29%29".format(str(tax_id))

tax_id_download = str(requests.get(url_tax_id_descendent).text)
tax_id_list = tax_id_download.split("\n")
tax_id_list.pop() #Remove the last element of the list (is empty)

tax_id_list = [int(taxid) for taxid in tax_id_list] #Convert the string into integers


# 2.2 taxID organism name

url_org_name = "https://rest.uniprot.org/taxonomy/stream?format=json&query=%28%28tax_id%3A{}%29%29".format(str(tax_id))

org_name_str = str(requests.get(url_org_name).text)
org_name_dic = json.loads(org_name_str)

org_name = org_name_dic["results"][0]["scientificName"]


#3. Convert JSON format to FASTA

def json_to_fasta(record, gene_name=0):

    #header creator
    header = ">" + record["uniParcCrossReferences"][0]["database"] + "|" + record["uniParcId"] + "|" + \
    record["uniParcCrossReferences"][0]["lastUpdated"]+ " "
    
    if "proteinName" in record["uniParcCrossReferences"][0]:
        header += record["uniParcCrossReferences"][0]["proteinName"] + " "

    if "organism" in record["uniParcCrossReferences"][0]:          
        if record["uniParcCrossReferences"][0]["organism"]["taxonId"] not in tax_id_list:
            header += "OS=" + org_name + " " + \
            "OX=" + str(tax_id) + " "
        else:
            header += "OS=" + record["uniParcCrossReferences"][0]["organism"]["scientificName"] + " " + \
            "OX=" + str(record["uniParcCrossReferences"][0]["organism"]["taxonId"]) + " "
                
    if gene_name != 0:
        header += "GN=" + gene_name
    elif gene_name == 0:
        if "geneName" in record["uniParcCrossReferences"][0]:
            header += "GN=" + record["uniParcCrossReferences"][0]["geneName"]

    #sequence creator
    sequence = record["sequence"]["value"]

    return (header, sequence)

#4. Programmatic acces to UniProtKb through the UniProt API  

#remove the previous database with the same name

if os.path.exists(db_name):
    os.remove(db_name)

#Programatic acces setup

re_next_link = re.compile(r'<(.+)>; rel="next"')
retries = Retry(total=5, backoff_factor=0.25, status_forcelist=[500, 502, 503, 504])
session = requests.Session()
session.mount("https://", HTTPAdapter(max_retries=retries))

def get_next_link(headers):
    if "Link" in headers:
        match = re_next_link.match(headers["Link"])
        if match:
            return match.group(1)

def get_batch(batch_url):
    while batch_url:
        response = session.get(batch_url)
        response.raise_for_status()
        total = response.headers["x-total-results"]
        yield response, total
        batch_url = get_next_link(response.headers)

#Iteratet throught the list of genes and create the db fasta file to append the protein records

if "gene_list_path" in globals():
    
    #API URL needed to download the proper sequences
    url_api = "https://rest.uniprot.org/uniparc/search?compressed=false&format=json&query=%28%28taxonomy_id%3A{}%29%20AND%20%28".format(str(tax_id))
    
    for gene in gene_list:
        url_api += "%28gene%3A{}%29".format(gene) #Gene indicator

        if gene != gene_list[-1]:
            url_api += "%20OR%20" #OR operator
        
    url_api += "%29%29&size=100" #End of the URL

progress = 0

with open(db_name, "wt") as file:
    for batch, total in get_batch(url_api):
        if int(total) > 0: #If there are more than one record found
            json_retrieve = batch.json()
            for protein in json_retrieve["results"]:
                retrive = json_to_fasta(protein)
                file.write(retrive[0] + "\n")
                        
                sequence_split = re.findall(".{1,60}", retrive[1])
                for row in sequence_split:
                    file.write(row + "\n")
            progress += len(json_retrieve["results"])
            print(f'{progress} / {total}')

In [44]:
#Metadata

import os
from datetime import datetime

#Global variables
database_path = "proboscidea_db.fasta"
metadata_path = "metadata.txt"

#Current time and date
now = datetime.now()
dat_time = now.strftime("%d/%m/%Y - %H:%M:%S")

#Calculate number of records and records with a gene
record_num = os.popen(f"grep -E -c '^>' {database_path}").read()
record_with_gene = os.popen(f"grep -E '^>' {database_path} | grep -c 'GN='").read()

warnings = 0

#Append the metadata information in a file
with open(metadata_path, "wt") as metadata:
    metadata.write("## METADATA for the database '{}'".format(database_path) + "\n" + \
                   "{}".format(dat_time) + "\n" + "\n")
                   
    metadata.write("# WARNINGS\n")
    if record_num != record_with_gene:
        warnings += 1
        metadata.write("Check gene names (GN=)\n")
    
    if warnings == 0:
        metadata.write("No one\n")

    metadata.write("\n# Stats about the protein records\n" + \
                    "number_of_records: {}".format(record_num) + \
                    "number_of_records_with_gene_(GN=): {}/{}".format(record_with_gene[:-1],record_num) + "\n")
                    
    metadata.write("# Stats about the genes found")

In [59]:
#Bash oneline

import os

database_stats_path = "example.csv"

os.system(f"echo 'Database,Count' > {database_stats_path}")
os.system("cut -f2 -d, metadata_proboscidea/records_info.csv | tail -n +2 | sort | uniq -c | while read line; do words=($line); echo ${words[1]},${words[0]}; done" + f"| sort -k 2 -n -r -t , >> {database_stats_path}")

0

In [ ]:
# New metadata

import os
import re
import shutil
import pandas as pd
import argparse
from datetime import datetime

# Script information - Written in Python 3.9.12 - May 2023
__author__ = "Guillermo Carrillo Martín"
__maintainer__ = "Guillermo Carrillo Martín"
__email__ = "guillermo.carrillo@upf.edu"

"""
This script reads a protein multi-fasta database and creates a folder with metadata 
information. The seven files created by the script are:

1. metadata_readme.txt; A text file that contains a summary about all the metadata
    retrieved from the database. It also shows the paths to all the other files.

2. records_info.csv; A csv file that contains all the information present in each 
    record header. To see a detailed description of the header information, see 
    the README.md file.

3. databases_employed.csv; A csv file that contains the number of the different
    sources (databases) used to build the multi-fasta. 

4. genes_retrieved.csv; A csv file that contains the number of genes retrieved.

5. genes_NOT_retrieved.txt; A text file that contains the genes not retrieved. This
    file is only created if the list of genes used to build the multi-fasta is 
    specified.

6. species_retrieved.csv; A csv file that contains the number of species retrieved, 
    indicating the scientific name and the TaxID.

7. species_genes.csv; A csv file that that shows the number of genes retrieved 
    for each species.

----------------------------------------------------------------------------------

The script has two main sections:

1. Calculate all the metadata information and stats from the multi-fasta. Also, 
    creates all the files previously mentioned except the metadata_readme.txt file.

2. Create the metadata_readme.txt file and print there all the information previously
    calculated. 
"""

#0. Set the arguments to run the program from the command line (parser)
parser = argparse.ArgumentParser(description="A script that creates metadata information for a database created by uniparc_db.py")
parser.add_argument("--input_path", dest="input_path", type=str, help="", required=True, nargs="?")
parser.add_argument("--output_folder", dest="folder_path", type=str, help="", required=True, nargs="?")
parser.add_argument("--genes", dest="gene_list", type=str, help="(nor required)", required=False, nargs="?")

args = parser.parse_args()

database_path = args.input_path
metadata_folder_path = args.folder_path
gene_list_path = args.gene_list

#1. Creating stats
#Create metadata folder
if os.path.exists(metadata_folder_path):
    shutil.rmtree(metadata_folder_path)

os.mkdir(metadata_folder_path)

#Current time and date
now = datetime.now()
dat_time = now.strftime("%d/%m/%Y - %H:%M:%S")

#Create the gene list if the path is specified
if gene_list_path != None:

    #a. Append in a list the genes in the gene list
    gene_list = []
    with open(gene_list_path, "rt") as gene_list_file: #Open gene list file
        for gene in gene_list_file:
            gene_list.append(gene[0:-1].strip())

#1.1 Stats about the protein records
#1.1.1 Number of records; records with gene name, scientfic name and TaxI
record_num = os.popen(f"grep -E -c '^>' {database_path}").read() 
record_with_gene = os.popen(f"cat {database_path} | grep -E '^>' | grep -c 'GN='").read()
record_with_taxname = os.popen(f"cat {database_path} | grep -E '^>' | grep -c 'OS='").read()
record_with_taxid = os.popen(f"cat {database_path} | grep -E '^>' | grep -c 'OX='").read()

#1.1.2 Create records_info.csv
#a. Function that retrieves a header tag value using a regular expression
def retrieve_tag(regular_expression, header):
    tag_match = re.search(r"" + regular_expression, header)
    
    if tag_match:
        tag = tag_match.group(1)
    
    elif not tag_match:
        tag = ""

    return tag

#b. Iterate through the multi-fasta 
with open(database_path, "rt") as database_file:
    records = [] #

    for line in database_file:  
        if line.startswith(">"): #For each header in the multi-fasta
            #Retrieve the values of the following parameters:
            
            upi = retrieve_tag("\|([^|]+)\|", line)  # Unic identifier
            database = retrieve_tag("^>([^|]+)", line) # Database
            gene = retrieve_tag("GN=(\w+)", line).upper() # Gene
            species = retrieve_tag("OS=(.*?)\sOX=", line) # Species
            taxid = retrieve_tag("OX=(\w+)", line) # TaxID
            update = retrieve_tag("\|([\d-]+)", line) # Update date

            header_info = "{},{},{},{},{},{}".format(upi,database,gene,species,taxid,update)
            records.append(header_info)

#c. Print all the information in a csv file        
with open(metadata_folder_path + "/records_info.csv", "wt") as records_info_file:
    records_info_file.write("Unic Identifier,Database,Gene,Species,TaxID,Last Update\n")
    for header in records:
        records_info_file.write(header + "\n")

#d. Read the record csv file as a Panda dataframe for future calculations
record_df = pd.read_csv("metadata_proboscidea/records_info.csv")

#1.2 Stats about the employed databases
databases_count_sr = record_df["Database"].value_counts() #A Panda serie counting the values of "Databases"
databases_count_sr.to_csv(metadata_folder_path + "/databases_employed.csv", index_label="Database", header=["Count"])

databases_num = databases_count_sr.count()

#1.3 Stats about the genes found
#1.3.1 genes retrieved
record_df["Gene"] = record_df["Gene"].str.upper()
gene_count_sr = record_df["Gene"].value_counts() #A Panda serie counting the values of "Databases"
gene_count_sr.to_csv(metadata_folder_path + "/genes_retrieved.csv", index_label="Gene", header=["Count"])

genes_num = gene_count_sr.count()

# 1.3.2 Genes not retrieved, if we are using a gene list
if gene_list_path != None:

    #a. Append in a list the genes found
    gene_found_ls = list(record_df["Gene"].unique())

    #b. Compare the genes found with the gene list and create genes not found list
    gene_not_found_ls = []
    for gene in gene_list:
        if gene not in gene_found_ls:
            gene_not_found_ls.append(gene)

    #c. Write the genes not retrieved in a text file
    with open(metadata_folder_path + "/genes_NOT_retrieved.txt", "wt") as genes_not_retrieved:
        for gene in gene_not_found_ls:
            genes_not_retrieved.write(gene + "\n")

#1.4 Stats about the species in the database
#1.4.1 Species retrieved
species_count_sr = record_df[["Species", "TaxID"]].value_counts() #A Panda serie counting the values of "Databases"
species_count_sr.to_csv(metadata_folder_path + "/species_retrieved.csv", index_label=["Species", "TaxID"], header=["Count"])

species_num = species_count_sr.count()

#1.4.2 Genes per species

species_genes = {}



with open(database_path, "rt") as database:
    species_genes_dic = {}

    for line in database:
        if line.startswith(">"):
            
            #1.6.2 Create species + genes
            specie_match = re.search(r"OS=(.*?)\sOX=", line)
            taxid_match = re.search(r"OX=(\w+)", line)
            gene_match = re.search(r"GN=(\w+)", line)
            
            if specie_match and taxid_match and gene_match:
                species = specie_match.group(1)
                taxid = taxid_match.group(1)
                gene = gene_match.group(1).upper()

                trio = "{},{},{},".format(species,taxid,gene)
                if trio not in species_genes_dic:
                    species_genes_dic[trio] = 1
                elif trio in species_genes_dic:
                    species_genes_dic[trio] += 1
    
    #A list with the species found
    species_found_ls = list(record_df["Species"].unique())


    #If there is a gene list, fill the genes not found in a specie with the gene list
    if gene_list_path != None:

        for species in species_found_ls:
            for gene in gene_list:
                if species + gene + "," not in species_genes_dic:
                    species_genes_dic[species + gene] = "0"  

    #If
    if gene_list_path == None:
        ""     

#1.6.2 Create species and genes file
with open(metadata_folder_path + "/species_genes.csv", "wt") as species_genes:
    species_genes.write("Species,TaxID,Gene,Count\n")
    for header, count in species_genes_dic.items():
        species_genes.write(header + str(count) + "\n")

#sort csv first by species name and then by count of genes
species_genes_table = pd.read_csv(metadata_folder_path + "/species_genes.csv")
species_genes_table.sort_values(["Species", "Count"], axis=0,ascending=[True, False], inplace=True)

species_genes_table.to_csv(metadata_folder_path + "/species_genes.csv", index=False)

#2. Append the metadata information in a file
with open(metadata_folder_path + "/metadata_readme.txt", "wt") as metadata:
    metadata.write("## METADATA for the database '{}'".format(database_path) + "\n" + \
                   "{}".format(dat_time) + "\n" + "\n")

    #2.1 WARNINGS

    warnings = 0 
                   
    metadata.write("# WARNINGS\n")
    if record_num != record_with_gene:
        warnings += 1
        metadata.write("Check gene names (GN=)\n")
    
    if record_num != record_with_taxname:
        warnings += 1
        metadata.write("Check taxa names (OS=)\n")

    if record_num != record_with_taxid:
        warnings += 1
        metadata.write("Check taxIDs (OX=)\n")

    if gene_list_path != None:
        if len(gene_found_ls) != len(gene_list):
            warnings += 1
            metadata.write("{} genes not found\n".format((len(gene_list)) - (len(gene_found_ls))))

    if warnings == 0:
        metadata.write("No warnings\n")

    #2.2 RECORDS STATS
    metadata.write("\n# Stats about the protein records\n" + \
                    "records: {}".format(record_num) + \
                    "records_with_gene: {}/{}".format(record_with_gene[:-1],record_num) + \
                    "records_with_taxname: {}/{}".format(record_with_taxname[:-1],record_num) + \
                    "records_with_taxid: {}/{}".format(record_with_taxid[:-1],record_num) + 
                    "all_records_table_path: {}\n".format(metadata_folder_path + "/records_info.csv"))

    #2.3 DATATBASE STATS
    metadata.write("\n# Stats about the employed databases\n" + \
                    "databases_employed: {}\n".format(databases_num) + \
                    "databases_employed_path: {}\n\n".format(metadata_folder_path + "/databases_employed.csv"))

    #2.4 GENE STATS  
    if gene_list_path != None:            
        metadata.write("# Stats about the genes found\n" + \
                        "genes_retrieved: {}/{}\n".format(genes_num,str(len(gene_list))) + \
                        "genes_retrieved_path: {}\n".format(metadata_folder_path + "/genes_retrieved.csv") + \
                        "genes_not_retrieved_path: {}\n".format(metadata_folder_path + "/genes_NOT_retrieved.txt"))
    
    elif gene_list_path == None:
        metadata.write("# Stats about the genes found\n" + \
                        "genes_retrieved: {}\n".format(genes_num) + \
                        "genes_retrieved_path: {}\n".format(metadata_folder_path + "/genes_retrieved.csv"))

    #2.5 SPECIES STATS
    metadata.write("\n# Stats about the species in the database\n" + \
                    "species_retrieved: {}\n".format(species_num) + \
                    "species_retrieved_path: {}\n".format(metadata_folder_path + "/species_retrieved.csv") + \
                    "species_and_genes_path: {}".format(metadata_folder_path + "/species_genes.csv")) 